In [10]:
import torch
import torch.nn as nn
import numpy as np
from sklearn import datasets #to generate regression datasets
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [11]:
bc = datasets.load_breast_cancer()
x, y = bc.data, bc.target

n_samples, n_features = x.shape

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=1234)

#scale, this gives our features 0 means and varians, recommended during logistic regression
sc = StandardScaler() #forstår ikke
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

x_train = torch.from_numpy(x_train.astype(np.float32))
x_test = torch.from_numpy(x_test.astype(np.float32))
y_train = torch.from_numpy(y_train.astype(np.float32))
y_test = torch.from_numpy(y_test.astype(np.float32))

y_train = y_train.view(y_train.shape[0], 1)
y_test = y_test.view(y_test.shape[0], 1)

In [12]:
class CustomLogisticRegression(nn.Module):
    def __init__(self, n_input_features):
        super(CustomLogisticRegression, self).__init__()
        self.linear = nn.Linear(n_input_features, 1)
        
    def forward(self, x):
        y_hat = torch.sigmoid(self.linear(x))
        return y_hat
    
model = CustomLogisticRegression(n_features)

In [13]:
learning_rate = 0.01
criterion = nn.BCELoss() #Binary cross entropy loss
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)

In [21]:
num_epochs = 100
for epoch in range(num_epochs):
    y_hat = model(x_train)
    loss = criterion(y_hat, y_train)
    
    loss.backward()
    
    optimizer.step()
    
    optimizer.zero_grad()
    
    if (epoch + 1) % 10 == 0:
        print(f'epoch: {epoch + 1}, loss = {loss.item():.4f}')
        

epoch: 10, loss = 0.1447
epoch: 20, loss = 0.1426
epoch: 30, loss = 0.1407
epoch: 40, loss = 0.1389
epoch: 50, loss = 0.1371
epoch: 60, loss = 0.1354
epoch: 70, loss = 0.1338
epoch: 80, loss = 0.1323
epoch: 90, loss = 0.1308
epoch: 100, loss = 0.1293


In [22]:
with torch.no_grad():
    y_hat = model(x_test)
    y_hat_cls = y_hat.round()
    acc = y_hat_cls.eq(y_test).sum() / float(y_test.shape[0]) #forstår jeg ikke
    print(f'accuracy = {acc:.4f}')

accuracy = 0.9298
